# Import packages and modules

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import labeled data set

In [2]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('Labeled_data.txt', header=None, delim_whitespace=True, skiprows=37) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()

-------------                      ID  SourceID    RAdeg     DEdeg       Per  \
0              ZTFJ000000.14+721413.7         2  0.00061  72.23716  0.299150   
1              ZTFJ000000.19+320847.2         3  0.00080  32.14645  0.287059   
2              ZTFJ000000.26+311206.3         4  0.00109  31.20176  0.362217   
3              ZTFJ000000.30+711634.1         5  0.00125  71.27616  0.268515   
4              ZTFJ000000.30+233400.5         6  0.00125  23.56682  0.269874   

-------------    R21  phi21            T0    gmag    rmag  ...  phi21_r  \
0              0.263  6.308  58388.255579  19.613  18.804  ...    6.308   
1              0.010  8.024  58280.478081  15.311  14.610  ...    8.024   
2              0.132  6.281  58283.461994  16.350  15.844  ...    6.281   
3              0.160  5.236  58657.423517  19.144  17.875  ...    5.236   
4              0.193  6.302  58437.268664  17.890  16.944  ...    6.302   

-------------   R2_g   R2_r  Amp_g  Amp_r  log(FAP_g)  log(FAP_r)  Type  \
0              0.624  0.873  0.540  0.438     -13.490     -27.331    EW   
1              0.940  0.977  0.219  0.197      -7.506     -10.079    EW   
2              0.951  0.960  0.233  0.226      -7.830      -9.245    EW   
3              0.363  0.623  0.173  0.154      -9.865     -22.037    EW   
4              0.910  0.976  0.373  0.352      -7.075      -8.819    EW   

-------------  Dmin_g  Dmin_r  
0               0.190   0.078  
1               0.020   0.017  
2               0.013   0.020  
3               0.000   0.005  
4               0.098   0.034  

[5 rows x 27 columns]

# Column explanations

In [3]:
header_pd.head(27)

-------- ------- ---- -------------  \
0     1- 22     A22  ---            ID   
1    24- 29      I6  ---      SourceID   
2    31- 39    F9.5  deg         RAdeg   
3    41- 49    F9.5  deg         DEdeg   
4    51- 61   F11.7    d           Per   
5    63- 68    F6.3  ---           R21   
6    70- 74    F5.3  ---         phi21   
7    76- 88   F13.7    d            T0   
8    90- 95    F6.3  mag          gmag   
9    97-102    F6.3  mag          rmag   
10  104-116   F13.7    d         Per_g   
11  118-130   F13.7    d         Per_r   
12  132-135      I4  ---         Num_g   
13  137-140      I4  ---         Num_r   
14  142-147    F6.3  ---         R21_g   
15  149-154    F6.3  ---         R21_r   
16  156-160    F5.3  ---       phi21_g   
17  162-166    F5.3  ---       phi21_r   
18  168-173    F6.3  ---          R2_g   
19  175-180    F6.3  ---          R2_r   
20  182-186    F5.3  mag         Amp_g   
21  188-192    F5.3  mag         Amp_r   
22  194-201    F8.3  [-]    log(FAP_g)   
23  203-210    F8.3  [-]    log(FAP_r)   
24  212-216      A5  ---          Type   
25  218-222    F5.3  mag        Dmin_g   
26  224-228    F5.3  mag        Dmin_r   

   -------------------------------------------  
0             ZTF variables catalog identifier  
1                   Internal source identifier  
2    Right Ascension in decimal degrees (J2000  
3       Declination in decimal degrees (J2000)  
4                                       Period  
5                           Ratio of a_2_/a_1_  
6                    The {phi}_2_ - 2*{phi}_1_  
7                HJD of Minimum, HJD-2,400,000  
8                    Mean ZTF g band magnitude  
9                    Mean ZTF r band magnitude  
10                 Period determined in g band  
11                 Period determined in r band  
12              Number of detections in g band  
13              Number of detections in r band  
14                Ratio of a_2_/a_1_ in g band  
15                Ratio of a_2_/a_1_ in r band  
16         The {phi}_2_ - 2*{phi}_1_ in g band  
17         The {phi}_2_ - 2*{phi}_1_ in r band  
18                          R-square in g band  
19                          R-square in r band  
20                         Amplitude in g band  
21                         Amplitude in r band  
22           False alarm probability in g band  
23           False alarm probability in r band  
24                               Variable type  
25                Minumum difference in g band  
26                Minumum difference in r band

In [ ]:
# Get complete light curve for each orow of labeled dataset

In [4]:
lightcurves = {} # empty dictionary to hold dataframe for each light curve
circle_radius = 0.00001 # 1 arcsec = 0.00028 degress
t_format = "ipac_table"
table_format = "FORMAT=" + str(t_format)
flag_mask = 32768
mask = "BAD_CATFLAGS_MASK=" + str(flag_mask)

for i in range(0,10):
# for i in range(0,len(labeled_data.index)):
    
    ra = labeled_data.RAdeg[i]
    dec = labeled_data.DEdeg[i]
    
    circle = "POS=CIRCLE"+"+"+str(ra)+"+"+str(dec)+"+"+str(circle_radius)
    
    params = circle + "&" + mask + "&" + table_format
    
    url= "https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?" + params
    data = pd.read_csv(url, header=None, delim_whitespace=True, skiprows=55) # extract data
    header = pd.read_csv(url, header=None, sep='|', skiprows=50,usecols=range(1,25), nrows=1)
    data.columns = header.iloc[0].str.strip()
    lightcurves [str(i)] = data
    print("row "+str(i) + (" unique objects extracted"))
    print(data.oid.unique())

row 0 unique objects extracted
[853213200018329]
row 1 unique objects extracted
[ 648107400006917 1645215400006764]
row 2 unique objects extracted
[648103100000752 648203100001041]
row 3 unique objects extracted
[ 833216400025183  833316400029428 1880205300017107]
row 4 unique objects extracted
[ 600103100002998  600203100005780  600303100006749 1596111200004823
 1596211200010461]
row 5 unique objects extracted
[ 806208300062497  832205400006687 1838113300052018 1838213300062832]
row 6 unique objects extracted
[832201400017963]
row 7 unique objects extracted
[806208200063685 832305100060634]
row 8 unique objects extracted
[805109200013762]
row 9 unique objects extracted
[ 806108200002352  806208200003594  832105100002021  832305100004179
 1809216200012620 1838113200037741 1838213200017590]


In [ ]:
print(url)

In [ ]:
labeled_data.head(10)

In [ ]:
df = lightcurves['0']

In [ ]:
df.oid.unique()

In [ ]:
print(len(df))